In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
import os
from xml.etree import ElementTree
from matplotlib import pyplot as plt
import tensorflow as tf
from sklearn.metrics import confusion_matrix
from tensorflow.keras import datasets, layers, models
keras = tf.keras
class_names = ['person','person-like']
class_names_label = {class_name:i for i, class_name in enumerate(class_names)}

n_classes = 2
size = (200,200)
import os
import cv2
import numpy as np
from xml.etree import ElementTree
def load_data():
    datasets = [
    "C:\\Users\\Aaditya Ahire\\Desktop\\sem6\\nullclass\\pedestrianDetection\\pedestrainDetect\\dataset\\Train\\Train",
    "C:\\Users\\Aaditya Ahire\\Desktop\\sem6\\nullclass\\pedestrianDetection\\pedestrainDetect\\dataset\\Test\\Test",
    "C:\\Users\\Aaditya Ahire\\Desktop\\sem6\\nullclass\\pedestrianDetection\\pedestrainDetect\\dataset\\Val\\Val"
]

    output = []

    for dataset in datasets:
        imags = []
        labels = []
        directoryA = os.path.join(dataset, 'Annotations')
        directoryIMG = os.path.join(dataset, 'JPEGImages')
        file = os.listdir(directoryA)
        img = os.listdir(directoryIMG)
        file.sort()
        img.sort()

        i = 0
        for xml in file:
            xmlf = os.path.join(directoryA, xml)
            dom = ElementTree.parse(xmlf)
            vb = dom.findall('object')
            label = vb[0].find('name').text
            labels.append(class_names_label[label])

            img_path = os.path.join(directoryIMG, img[i])
            curr_img = cv2.imread(img_path)
            # Assuming 'size' is defined somewhere in your code
            curr_img = cv2.resize(curr_img, size)
            imags.append(curr_img)
            i += 1
        
        imags = np.array(imags, dtype='float32')
        imags = imags / 255
        
        labels = np.array(labels, dtype='int32')

        output.append((imags, labels))
    
    return output


(train_images, train_labels),(test_images, test_labels),(val_images, val_labels) = load_data()
train_images.shape

from tensorflow.keras import models
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import numpy as np

# Assume you have already loaded and preprocessed train_images, train_labels, test_images, and test_labels

model = models.Sequential()
model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(200, 200, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(2))

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.summary()

history = model.fit(train_images, train_labels, epochs=20, validation_data=(test_images, test_labels))

# Save the model
model.save("action.h5")





C:\Users\Aaditya Ahire\AppData\Local\Temp\ipykernel_20980\3355540971.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)






Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 198, 198, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2  (None, 99, 99, 64)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 97, 97, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 48, 48, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 46, 46, 128)       73856     
                                                                 
 flatten (Flatten)           (None, 270848)         

C:\Users\Aaditya Ahire\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [2]:

# Load the saved model
loaded_model = models.load_model("action.h5")

In [6]:
train_accuracy = history.history['accuracy'][-1]
print(f"Final Accuracy: {train_accuracy:.4f}")


Final Accuracy: 0.9989


In [7]:
import cv2
import numpy as np
from tensorflow.keras import models

# Load the saved model
loaded_model = models.load_model("action.h5")

# Input an image from PC
input_image_path = "image_test.jpeg"
input_image = cv2.imread(input_image_path)
size = (200, 200)
input_image_resized = cv2.resize(input_image, size)
input_image_normalized = input_image_resized / 255.0
input_image_expanded = np.expand_dims(input_image_normalized, axis=0)

# Make Predictions
predictions = loaded_model.predict(input_image_expanded)
predicted_class_index = np.argmax(predictions)
predicted_class = class_names[predicted_class_index]

# Display output message
if predicted_class == 'person':
    print("Pedestrian detected.")
else:
    print("No pedestrian detected.")



1/1 [==============================] - 0s 132ms/step
Pedestrian detected.


In [8]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import cv2
import numpy as np

class PedestrianDetectorGUI:
    def __init__(self, model):
        self.model = model

        # Initialize Tkinter window with a larger size
        self.root = tk.Tk()
        self.root.title("Pedestrian Detector")
        self.root.geometry("400x400")  # Set the initial window size

        # Create and configure the main frame
        self.main_frame = tk.Frame(self.root, bg='#F0F0F0')  # Set background color
        self.main_frame.pack(fill=tk.BOTH, expand=True)

        # Create labels for displaying image and predicted class
        self.image_label = tk.Label(self.main_frame, bg='#FFFFFF', relief=tk.SOLID, borderwidth=2)
        self.image_label.pack(pady=10)

        self.predicted_class_label = tk.Label(self.main_frame, text="Predicted Class: ", font=("Helvetica", 12), bg='#F0F0F0')
        self.predicted_class_label.pack(pady=5)

        # Create a button to load an image
        self.load_button = tk.Button(self.main_frame, text="Load Image", command=self.load_image, relief=tk.RAISED, borderwidth=2)
        self.load_button.pack(pady=10)

        # Run the Tkinter main loop
        self.root.mainloop()

    def load_image(self):
        # Open a file dialog to select an image file
        file_path = filedialog.askopenfilename()

        if file_path:
            # Load and preprocess the selected image
            input_image = cv2.imread(file_path)
            input_image_resized = cv2.resize(input_image, size)
            input_image_normalized = input_image_resized / 255.0
            input_image_expanded = np.expand_dims(input_image_normalized, axis=0)

            # Make predictions
            predictions = self.model.predict(input_image_expanded)
            predicted_class_index = np.argmax(predictions)
            predicted_class = class_names[predicted_class_index]

            # Display the input image
            self.display_image(input_image)

            # Update the predicted class label
            self.predicted_class_label.config(text=f"Predicted Class: {predicted_class}")

            # Show detection result directly on the GUI window
            if predicted_class == 'person':
                
                detection_result = "Pedestrian detected!" 
            else:
                detection_result="No pedestrian detected."
            self.display_detection_result(detection_result)

    def display_image(self, image):
        # Convert the image to RGB format
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Convert the image to a PhotoImage object
        image_tk = ImageTk.PhotoImage(Image.fromarray(image_rgb))

        # Update the image label
        self.image_label.config(image=image_tk)
        self.image_label.image = image_tk  # Keep a reference to avoid garbage collection issues

    def display_detection_result(self, result):
        # Update the GUI window with the detection result
        result_label = tk.Label(self.main_frame, text=result, font=("Helvetica", 14, "bold"), fg='#228B22' if "Pedestrian" in result else '#FF0000', bg='#F0F0F0')
        result_label.pack(pady=10)

if __name__ == "__main__":
    # Create an instance of the PedestrianDetectorGUI class and pass the loaded model
    gui = PedestrianDetectorGUI(loaded_model)


1/1 [==============================] - 0s 50ms/step
